In [23]:
# IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
import geopandas as gpd
import numpy as np
from shapely.geometry import box
from typing import Tuple, Dict, Any, List
import panel as pn
from matplotlib.figure import Figure
from tkinter import Tk
from tkinter.filedialog import askopenfilenames
from tkinter import Tk, Toplevel, Button, Checkbutton, IntVar, Label, Frame, filedialog
import cupy as cp
import easygui
import ipywidgets as widgets
from IPython.display import display
from joblib import Parallel, delayed\



In [45]:
#FUNCTION DEFINITION - CPU (only works for geol layer)

def geojson_to_numpy_grid_3d_batch(
    grid_size: Tuple[int, int],  # Grid size for the output array
    target_crs: str = "EPSG:3857"  # Web Mercator projection
) -> Tuple[np.ndarray, Dict[str, np.ndarray], Dict[str, Dict[Any, int]], List[Dict[str, Any]]]:
    # Open a file selection dialog for the user to select multiple GeoJSON files
    root = Tk()
    root.withdraw()  # Hide the root window
    root.attributes("-topmost", True)  # Bring the dialog to the front

    # Open the file selection dialog
    geojson_files = askopenfilenames(
        title="Select GeoJSON Files",
        filetypes=[("GeoJSON files", "*.geojson"), ("All files", "*.*")]
    )

    all_feature_grids = {}
    all_feature_mappings = {}
    geospatial_info_list = []

    for geojson_file in geojson_files:
        # Read the GeoJSON file
        gdf = gpd.read_file(geojson_file)

        # Reproject to target CRS
        gdf = gdf.to_crs(target_crs)

        # Get the total bounds of all geometries
        minx, miny, maxx, maxy = gdf.total_bounds

        # Create a fixed-size grid
        x = np.linspace(minx, maxx, grid_size[1] + 1)
        y = np.linspace(miny, maxy, grid_size[0] + 1)

        # Automatically extract all relevant feature columns, excluding geometry columns
        feature_columns = [col for col in gdf.columns if col != gdf.geometry.name]

        # Get the filename without extension for prefixing
        filename_prefix = os.path.splitext(os.path.basename(geojson_file))[0]

        # Dictionary to hold grids and category mappings for each feature column
        feature_grids = {}
        feature_mappings = {}

        # Store geospatial information for each file
        geospatial_info = {
            'transform': (minx, miny, maxx, maxy),
            'crs': target_crs,
            'file_name': filename_prefix
        }
        geospatial_info_list.append(geospatial_info)

        # Iterate over each feature column
        for feature_column in feature_columns:
            # Get unique categories and create a mapping to integers
            unique_categories = gdf[feature_column].unique()
            
            # Prefix each feature class with the filename
            category_to_int = {f"{filename_prefix}_{cat}": i for i, cat in enumerate(unique_categories)}

            # Initialize the 2D NumPy array with -1 (representing no data)
            grid = np.full(grid_size, -1, dtype=int)

            # Create a spatial index for faster intersection checks
            sindex = gdf.sindex

            # Pre-compute cell geometries
            cells = [box(x[j], y[i], x[j + 1], y[i + 1])
                     for i in range(grid_size[0])
                     for j in range(grid_size[1])]

            # Vectorized operations for intersection
            def process_cell(cell, possible_matches):
                if possible_matches.empty:
                    return -1
                intersections = possible_matches.geometry.intersection(cell)
                intersection_areas = intersections.area
                largest_intersection_idx = intersection_areas.idxmax()
                category = possible_matches.loc[largest_intersection_idx, feature_column]
                return category_to_int[f"{filename_prefix}_{category}"]

            # Iterate through each cell in the grid
            for idx, cell in enumerate(cells):
                i, j = divmod(idx, grid_size[1])

                # Use the spatial index to find potential intersecting polygons
                possible_matches_index = list(sindex.intersection(cell.bounds))
                if not possible_matches_index:
                    continue

                # Check for actual intersection and assign the feature value
                possible_matches = gdf.iloc[possible_matches_index]
                grid[i, j] = process_cell(cell, possible_matches)

            # Store the grid and category mapping for this feature
            feature_grids[f"{filename_prefix}_{feature_column}"] = grid
            feature_mappings[f"{filename_prefix}_{feature_column}"] = category_to_int

        # Merge with all feature grids and mappings
        all_feature_grids.update(feature_grids)
        all_feature_mappings.update(feature_mappings)

    # Stack all grids into a 3D array
    grid_3d = np.stack(list(all_feature_grids.values()), axis=0)

    return grid_3d, all_feature_grids, all_feature_mappings, geospatial_info_list


In [10]:
# PARALLEL PROCESSING FUNCTION - CPU - HYBRID STRATEGY

def process_cell(idx, cell, gdf, sindex, feature_column, category_to_int, filename_prefix):
    """
    Process a single cell by finding intersections and determining the feature value.
    """
    i, j = divmod(idx, grid_size[1])

    # Use the spatial index to find potential intersecting polygons
    possible_matches_index = list(sindex.intersection(cell.bounds))
    if not possible_matches_index:
        return i, j, np.nan  # No intersecting features, return NaN for no data

    # Check for actual intersection and assign the feature value
    possible_matches = gdf.iloc[possible_matches_index]
    if possible_matches.empty:
        return i, j, np.nan

    # Calculate intersections more precisely
    intersections = possible_matches.geometry.intersection(cell)

    # Consider all non-zero intersections
    valid_intersections = intersections[intersections.area > 0]

    if valid_intersections.empty:
        return i, j, np.nan

    # Determine strategy based on the number of intersecting features
    if len(valid_intersections) > 5:  # Threshold for choosing strategy
        # Many small polygons: Sum the areas for each unique category
        areas_per_category = {}
        for idx, intersection in enumerate(valid_intersections):
            if not intersection.is_empty:
                category = possible_matches.iloc[idx][feature_column]
                category_key = f"{filename_prefix}_{category}"
                if category_key not in areas_per_category:
                    areas_per_category[category_key] = 0
                areas_per_category[category_key] += intersection.area

        # Choose the category with the largest cumulative area
        if areas_per_category:
            max_category = max(areas_per_category, key=areas_per_category.get)
            return i, j, category_to_int[max_category]
        else:
            return i, j, np.nan  # Fallback to NaN

    else:
        # Few large polygons: Choose the largest single intersection by area
        largest_intersection_idx = valid_intersections.area.idxmax()
        category = possible_matches.loc[largest_intersection_idx, feature_column]
        return i, j, category_to_int[f"{filename_prefix}_{category}"]

def process_feature_column(geojson_file, feature_column, grid_size, target_crs, filename_prefix, x, y):
    # Read the GeoJSON file
    gdf = gpd.read_file(geojson_file)

    # Reproject to target CRS
    gdf = gdf.to_crs(target_crs)

    # Get unique categories and create a mapping to integers
    unique_categories = gdf[feature_column].unique()
    category_to_int = {f"{filename_prefix}_{cat}": i for i, cat in enumerate(unique_categories)}

    # Initialize the 2D NumPy array with NaN (representing no data)
    grid = np.full(grid_size, np.nan)

    # Create a spatial index for faster intersection checks
    sindex = gdf.sindex

    # Pre-compute cell geometries
    cells = [box(x[j], y[i], x[j + 1], y[i + 1])
             for i in range(grid_size[0])
             for j in range(grid_size[1])]

    # Use joblib to parallelize cell processing
    results = Parallel(n_jobs=-1)(delayed(process_cell)(
        idx, cell, gdf, sindex, feature_column, category_to_int, filename_prefix
    ) for idx, cell in enumerate(cells))

    # Fill the grid with the results
    for i, j, value in results:
        grid[i, j] = value

    # Return the grid and category mapping for this feature
    return (f"{filename_prefix}_{feature_column}", grid, category_to_int)

def geojson_to_numpy_grid_3d_batch(
    grid_size: Tuple[int, int],  # Grid size for the output array
    target_crs: str = "EPSG:3857"  # Web Mercator projection
) -> Tuple[np.ndarray, Dict[str, np.ndarray], Dict[str, Dict[Any, int]], List[Dict[str, Any]]]:
    # Open a file selection dialog for the user to select multiple GeoJSON files
    root = Tk()
    root.withdraw()  # Hide the root window
    root.attributes("-topmost", True)  # Bring the dialog to the front

    # Open the file selection dialog
    geojson_files = askopenfilenames(
        title="Select GeoJSON Files",
        filetypes=[("GeoJSON files", "*.geojson"), ("All files", "*.*")]
    )

    all_feature_grids = {}
    all_feature_mappings = {}
    geospatial_info_list = []

    results = []

    for geojson_file in geojson_files:
        # Read the GeoJSON file to get the total bounds
        gdf = gpd.read_file(geojson_file)
        gdf = gdf.to_crs(target_crs)
        minx, miny, maxx, maxy = gdf.total_bounds

        # Create a fixed-size grid
        x = np.linspace(minx, maxx, grid_size[1] + 1)
        y = np.linspace(miny, maxy, grid_size[0] + 1)

        # Automatically extract all relevant feature columns, excluding geometry columns
        feature_columns = [col for col in gdf.columns if col != gdf.geometry.name]

        # Get the filename without extension for prefixing
        filename_prefix = os.path.splitext(os.path.basename(geojson_file))[0]

        # Store geospatial information for each file
        geospatial_info = {
            'transform': (minx, miny, maxx, maxy),
            'crs': target_crs,
            'file_name': filename_prefix
        }
        geospatial_info_list.append(geospatial_info)

        # Use joblib to parallelize the processing of each feature column
        results.extend(Parallel(n_jobs=-1)(delayed(process_feature_column)(
            geojson_file, feature_column, grid_size, target_crs, filename_prefix, x, y
        ) for feature_column in feature_columns))

    # Process results to merge grids and mappings
    for feature_name, grid, category_to_int in results:
        all_feature_grids[feature_name] = grid
        all_feature_mappings[feature_name] = category_to_int

    # Stack all grids into a 3D array
    grid_3d = np.stack(list(all_feature_grids.values()), axis=0)

    return grid_3d, all_feature_grids, all_feature_mappings, geospatial_info_list


In [24]:
# PARALLEL PROCESSING - CPU - HYBRID - FEATURE SELECTION ENABLED (broken in ipynb)

# Function to prompt the user to select features using tkinter
def select_features(gdf_dict):
    """
    Create a tkinter GUI window to allow the user to select which features to process.
    """
    selected_features = {}
    root = Tk()
    root.withdraw()  # Hide the root window
    root.attributes("-topmost", True)  # Bring the dialog to the front

    # Function to create the selection window
    def create_selection_window():
        selection_window = Toplevel(root)
        selection_window.title("Select Features to Process")

        # Store variables for checkboxes
        feature_vars = {}

        # Create checkboxes for each file and its features
        for geojson_file, gdf in gdf_dict.items():
            feature_columns = [col for col in gdf.columns if col != gdf.geometry.name]

            # Extract just the file name from the full path
            file_name = os.path.basename(geojson_file)

            # Add a frame for each file
            frame = Label(selection_window, text=f"File: {file_name}")
            frame.pack(anchor='w', padx=10, pady=5)

            # Create checkboxes for each feature in the file
            feature_vars[geojson_file] = {}
            for feature in feature_columns:
                var = IntVar()
                checkbutton = Checkbutton(selection_window, text=feature, variable=var)
                checkbutton.pack(anchor='w')
                feature_vars[geojson_file][feature] = var

        # Function to handle "OK" button click
        def on_ok():
            for geojson_file, features in feature_vars.items():
                selected_features[geojson_file] = [feature for feature, var in features.items() if var.get() == 1]
            print("Selected Features Debugging:", selected_features)  # Debugging: print selected features
            if all(not features for features in selected_features.values()):
                print("No features were selected.")  # Debugging: Check if no features were selected
            selection_window.destroy()
            root.quit()  # Properly close the Tkinter main loop

        # Add an "OK" button
        Button(selection_window, text="OK", command=on_ok).pack(pady=10)

        selection_window.mainloop()

    create_selection_window()
    root.destroy()  # Ensure the root window is properly destroyed
    print("Final Selected Features Debugging:", selected_features)  # Debugging: print final selected features
    return selected_features

# Function to process cells in chunks
def process_chunk(chunk, gdf, sindex, feature_column, category_to_int, filename_prefix):
    results = []
    for idx, cell in chunk:
        i, j = divmod(idx, grid_size[1])
        possible_matches_index = list(sindex.intersection(cell.bounds))
        if not possible_matches_index:
            results.append((i, j, np.nan))
            continue

        possible_matches = gdf.iloc[possible_matches_index]
        if possible_matches.empty:
            results.append((i, j, np.nan))
            continue

        intersections = possible_matches.geometry.intersection(cell)
        valid_intersections = intersections[intersections.area > 0]

        if valid_intersections.empty:
            results.append((i, j, np.nan))
            continue

        if len(valid_intersections) > 5:
            areas_per_category = {}
            for idx, intersection in enumerate(valid_intersections):
                if not intersection.is_empty:
                    category = possible_matches.iloc[idx][feature_column]
                    category_key = f"{filename_prefix}_{category}"
                    if category_key not in areas_per_category:
                        areas_per_category[category_key] = 0
                    areas_per_category[category_key] += intersection.area

            if areas_per_category:
                max_category = max(areas_per_category, key=areas_per_category.get)
                results.append((i, j, category_to_int[max_category]))
            else:
                results.append((i, j, np.nan))
        else:
            largest_intersection_idx = valid_intersections.area.idxmax()
            category = possible_matches.loc[largest_intersection_idx, feature_column]
            results.append((i, j, category_to_int[f"{filename_prefix}_{category}"]))
    return results

# Function to process each feature column
def process_feature_column(gdf, feature_column, grid_size, target_crs, filename_prefix, x, y):
    gdf = gdf.to_crs(target_crs)

    unique_categories = gdf[feature_column].unique()
    category_to_int = {f"{filename_prefix}_{cat}": i for i, cat in enumerate(unique_categories)}

    grid = np.full(grid_size, np.nan)

    sindex = gdf.sindex

    cells = [(idx, box(x[j], y[i], x[j + 1], y[i + 1]))
             for idx, (i, j) in enumerate(np.ndindex(grid_size))]

    chunk_size = max(1, len(cells) // 10)
    chunks = [cells[i:i + chunk_size] for i in range(0, len(cells), chunk_size)]

    results = Parallel(n_jobs=-1, backend='loky')(delayed(process_chunk)(
        chunk, gdf, sindex, feature_column, category_to_int, filename_prefix
    ) for chunk in chunks)

    for result in results:
        for i, j, value in result:
            grid[i, j] = value

    return (f"{filename_prefix}_{feature_column}", grid, category_to_int)

# Main function
def geojson_to_numpy_grid_3d_batch(
    grid_size: Tuple[int, int],  # Grid size for the output array
    target_crs: str = "EPSG:3857"  # Web Mercator projection
) -> Tuple[np.ndarray, Dict[str, np.ndarray], Dict[str, Dict[Any, int]], List[Dict[str, Any]]]:
    # Use tkinter to select files
    root = Tk()
    root.withdraw()  # Hide the root window
    root.attributes("-topmost", True)  # Bring the dialog to the front
    geojson_files = filedialog.askopenfilenames(
        title="Select GeoJSON Files",
        filetypes=[("GeoJSON files", "*.geojson"), ("All files", "*.*")]
    )
    root.destroy()

    if not geojson_files:
        raise ValueError("No files selected. Please select at least one GeoJSON file.")

    # Load all selected files into GeoDataFrames
    gdf_dict = {geojson_file: gpd.read_file(geojson_file) for geojson_file in geojson_files}

    # Ask the user to select features to process
    selected_features = select_features(gdf_dict)

    all_feature_grids = {}
    all_feature_mappings = {}
    geospatial_info_list = []

    results = []

    for geojson_file in geojson_files:
        gdf = gdf_dict[geojson_file]
        gdf = gdf.to_crs(target_crs)
        minx, miny, maxx, maxy = gdf.total_bounds

        x = np.linspace(minx, maxx, grid_size[1] + 1)
        y = np.linspace(miny, maxy, grid_size[0] + 1)

        feature_columns = selected_features.get(geojson_file, [])

        if not feature_columns:
            print(f"No features selected for file: {geojson_file}")
            continue

        filename_prefix = os.path.splitext(os.path.basename(geojson_file))[0]

        geospatial_info = {
            'transform': (minx, miny, maxx, maxy),
            'crs': target_crs,
            'file_name': filename_prefix
        }
        geospatial_info_list.append(geospatial_info)

        results.extend(Parallel(n_jobs=-1)(delayed(process_feature_column)(
            gdf, feature_column, grid_size, target_crs, filename_prefix, x, y
        ) for feature_column in feature_columns))

    for feature_name, grid, category_to_int in results:
        all_feature_grids[feature_name] = grid
        all_feature_mappings[feature_name] = category_to_int

    if not all_feature_grids:
        raise ValueError("No features were processed. Please select at least one feature to process.")

    grid_3d = np.stack(list(all_feature_grids.values()), axis=0)

    return grid_3d, all_feature_grids, all_feature_mappings, geospatial_info_list


In [28]:
# COMPUTE GRID SIZE

def compute_grid_size(geojson_file: str, short_edge_cells: int = 1200) -> Tuple[int, int]:
    # Read the GeoJSON file
    gdf = gpd.read_file(geojson_file)
    
    # Get the bounding box of the masking region
    minx, miny, maxx, maxy = gdf.total_bounds
    
    # Calculate width and height of the bounding box
    width = maxx - minx
    height = maxy - miny

    # Determine which is the short and long edge
    if width < height:
        short_edge = width
        long_edge = height
        orientation = 'portrait'
    else:
        short_edge = height
        long_edge = width
        orientation = 'landscape'

    # Compute the aspect ratio
    aspect_ratio = long_edge / short_edge

    # Compute the number of cells for the long edge
    long_edge_cells = int(short_edge_cells * aspect_ratio)

    # Determine the grid size based on the orientation
    if orientation == 'portrait':
        grid_size = (short_edge_cells, long_edge_cells)
    else:
        grid_size = (long_edge_cells, short_edge_cells)

    return grid_size

mask_file = r"C:\Users\TyHow\Documents\3. Work\GIS Stuff\ML_pilot_data\MASK.geojson"
grid_size = compute_grid_size(mask_file, short_edge_cells=10)[::-1]
print(f"Calculated grid size: {grid_size}")


Calculated grid size: (12, 10)


In [31]:
#RUN FUNCTION

#grid_size = (1200, 1550)  # Define the grid size

# Call the function
grid_3d, feature_grids, feature_mappings, geospatial_info_list = geojson_to_numpy_grid_3d_batch(grid_size)

# Print results
print("Shape of the 3D grid array:", grid_3d.shape)
print("Feature grids:", feature_grids.keys())
print("Feature mappings:", feature_mappings)
print("Geospatial information for each file:", geospatial_info_list)


No features selected for file: C:/Users/TyHow/Documents/3. Work/GIS Stuff/ML_pilot_data/geology_clipped.geojson


ValueError: No features were processed. Please select at least one feature to process.

Selected Features Debugging: {'C:/Users/TyHow/Documents/3. Work/GIS Stuff/ML_pilot_data/geology_clipped.geojson': ['CODIGO', 'DEFINICION', 'GEOCHRON_A']}


In [9]:
#PLOT

# Initialize the Panel extension
pn.extension()

# Function to plot a specific layer using Matplotlib
def plot_layer_bokeh(layer_index):
    # Debugging: Print information about the current layer being plotted
    print(f"Plotting Layer {layer_index + 1}/{grid_3d.shape[0]}: {list(feature_grids.keys())[layer_index]}")
    print(f"Min value in layer: {np.min(grid_3d[layer_index])}, Max value in layer: {np.max(grid_3d[layer_index])}")

    # Create the plot
    fig = Figure(figsize=(3, 4))
    ax = fig.add_subplot(111)
    im = ax.imshow(grid_3d[layer_index], cmap='tab20', interpolation='nearest', aspect='auto')
    ax.set_title(f"Layer {layer_index + 1}: {list(feature_grids.keys())[layer_index]}")
    fig.colorbar(im, ax=ax, label='Classes')
    ax.set_xlabel('X Coordinate')
    ax.set_ylabel('Y Coordinate')

    # Debugging: Display the array data for the current layer
    print("Layer data:\n", grid_3d[layer_index])

    return pn.pane.Matplotlib(fig, tight=True)

# Create a Panel widget for selecting the layer
layer_slider = pn.widgets.IntSlider(name='Layer Index', start=0, end=grid_3d.shape[0] - 1, step=1, value=0)

# Bind the plotting function to the slider value
panel = pn.bind(plot_layer_bokeh, layer_index=layer_slider)

# Display the Panel with the slider and plot
pn.Column(layer_slider, panel).servable()


Plotting Layer 1/2: geology_clipped_fid
Min value in layer: 0.0, Max value in layer: 173.0
Layer data:
 [[161. 161. 126. 170. 170. 170. 170. 137.   1.  70.]
 [161. 163. 117. 170. 121. 170. 137. 137. 172.  63.]
 [163. 161. 161. 170. 173. 159. 159.  51. 167.  38.]
 [163. 163. 111. 111. 114.  55. 159.  51.  38.  38.]
 [105. 105. 111. 151. 159. 159. 159.   1. 110.  46.]
 [105. 151. 151. 151.   1. 155. 159.  43.  26. 101.]
 [157. 151. 151. 151. 155.  98. 102.  20. 101. 101.]
 [ 90. 151. 151.  90.  90.  98. 152.  20.  92.  92.]
 [ 90.  90.  85.  85.  85. 142. 142.  14.  88.  89.]
 [ 82.  82.  82.  82.  90. 142.   3.   9.   3. 144.]
 [ 82.  82. 138. 138. 138.   7.   0.   3. 141. 140.]
 [ 82.  82.  82. 138. 138. 138.   0.   0.   0.   0.]]


BokehModel(combine_events=True, render_bundle={'docs_json': {'b68800e6-e979-4e43-abaf-58a7a1521551': {'version…

Plotting Layer 2/2: possible_anomalies_clipped_fid
Min value in layer: nan, Max value in layer: nan
Layer data:
 [[nan nan nan nan nan nan nan nan nan  0.]
 [nan nan nan nan nan nan nan nan  4.  4.]
 [nan nan nan nan nan nan nan nan  4.  4.]
 [nan nan nan nan nan nan nan nan  4.  4.]
 [nan 20. 20. nan nan nan nan nan  5.  5.]
 [20. 20. 20. nan 13. 13. nan nan nan nan]
 [20. nan nan nan 13. 13. nan nan  9. nan]
 [nan nan nan nan nan 13. nan  7.  6. nan]
 [nan nan nan nan nan 14. nan nan nan nan]
 [nan nan 17. 16. 16. 19. 10. nan nan nan]
 [nan nan 17. 17. 18. 18. 21. 11. 11. 11.]
 [nan nan 17. 17. 18. 15. 21. 11. 11. nan]]
Plotting Layer 1/2: geology_clipped_fid
Min value in layer: 0.0, Max value in layer: 173.0
Layer data:
 [[161. 161. 126. 170. 170. 170. 170. 137.   1.  70.]
 [161. 163. 117. 170. 121. 170. 137. 137. 172.  63.]
 [163. 161. 161. 170. 173. 159. 159.  51. 167.  38.]
 [163. 163. 111. 111. 114.  55. 159.  51.  38.  38.]
 [105. 105. 111. 151. 159. 159. 159.   1. 110.  46.]
 

In [ ]:

# Save the array to a file
np.save("C:\Users\TyHow\Documents\3. Work\ML_test_area\exports\export_3d.npy", grid_3d)
